In [83]:
import json
import logging
from pathlib import Path
import re

from arcgis.geometry import Geometry
from tqdm.notebook import tqdm

from reach_tools.utils.procure import download_raw_json_from_aw

In [84]:
dir_data = Path.cwd().parent / 'data'
dir_raw = dir_data / 'raw'
dir_raw_aw = dir_raw / 'american_whitewater'

dir_raw_aw

PosixPath('/Users/joel5174/Library/Mobile Documents/com~apple~CloudDocs/projects/reach-tools/data/raw/american_whitewater')

In [85]:
logging.basicConfig(level="INFO")

In [86]:
if not dir_raw_aw.exists():
    dir_raw_aw.mkdir(parents=True)

In [87]:
fail_count = 0
max_fail = 1000
max_range = 10000000

In [ ]:
existing_reach_id_lst = [int(re.search(r"aw_(\d+)\.json", pth.name).group(1)) for pth in dir_raw_aw.glob('aw_*.json')]

for reach_id in range(1, max_range):

    # location for saving the reach json
    file_pth = dir_raw_aw / f'aw_{reach_id:08d}.json'

    # handle if file already exists
    if reach_id in existing_reach_id_lst:
        logging.debug(f"reach_id={reach_id} already exists at {file_pth}")

    else:        
        logging.debug(f"Attempting to download reach_id={reach_id}")
        
        try:
            # download the data from AW
            aw_json = download_raw_json_from_aw(reach_id)
    
            # if the file exists, clean it out
            if file_pth.exists():
                file_pth.unlink()
    
            # save to a local file
            with open(file_pth, 'x') as fp:
                json.dump(aw_json, fp, indent=2)
    
            fail_count = 0
    
            logging.info(f'Downloaded reach_id={reach_id} and saved to {file_pth}')
    
        except:
            fail_count += 1
            logging.debug(f'Could not retrieve data for reach_id={reach_id} (fail_count: {fail_count})')
            
            if fail_count > max_fail:
                logging.warning(f'It appears there is not any data for reach_id={reach_id}.')
                break

In [51]:
aw_id = int(re.search(r"aw_(\d+)\.json", pth.name).group(1))

aw_id

1372

In [ ]:
reach_id, json_pth = 

In [58]:
with open(pth, 'r') as json_fl:
    aw_json = json.load(json_fl)

json_info = aw_json.get('CContainerViewJSON_view').get('CRiverMainGadgetJSON_main').get('info')

reach_id = json_info.get('id')
river = json_info.get('river')
name = json_info.get('section')
description = json_info.get('description_md')

In [67]:
geojson = json_info.get('geom')

In [68]:
geometry = Geometry(geojson)

In [70]:
esri_json = geometry.JSON